# Daily Performance
Identify whether it is volatility or price movement that needs improvement

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error

In [36]:
file = "dailyPerf_2022-04-02_2022-04-05.csv"

In [37]:
df = pd.read_csv(file)
df.head()

,Date,Symbol,Exchange,garch,svr,mlp,LSTM,prev_Close,prediction,volatility,ActualDate,ActualClose,ActualPercent,ActualTrend
0,2022-04-04,AAL,NASDAQ,9.28532,2.47927,2.28121,16.6536,18.23,-1,4.68193,2022-04-05,17.84,2.139331,-1
1,2022-04-04,AAPL,NASDAQ,2.79687,1.75485,1.60878,159.0210,178.44,-1,2.05350,2022-04-05,175.06,1.894194,0
2,2022-04-04,AMAT,NASDAQ,6.81057,3.11612,3.11230,131.5450,129.44,1,4.34633,2022-04-05,121.71,5.971879,-1
3,2022-04-04,AMC,NYSE,101.91800,5.43978,5.71697,16.0923,23.31,-1,37.69170,2022-04-05,21.21,9.009009,-1
4,2022-04-04,AMD,NASDAQ,16.30120,4.39617,4.43730,110.4540,110.53,-1,8.37822,2022-04-05,106.82,3.356555,-1


In [38]:
df['LSTM_vol'] = (df.LSTM - df.prev_Close) / df.prev_Close * 100

## Which Voltatily metrics is closest

In [39]:
mean_squared_error(df.garch, df.ActualPercent)

190.61142176108277

In [40]:
mean_squared_error(df.svr, df.ActualPercent)

3.2556234171227607

In [41]:
mean_squared_error(df.mlp, df.ActualPercent)

2.8191002436477577

In [42]:
mean_squared_error(df.LSTM_vol, df.ActualPercent)

142.02571278119512

## New Aggregation

In [68]:
df["new_vol"] = df[['svr', 'mlp']].mean(axis=1) # ML

In [69]:
mean_squared_error(df.new_vol, df.ActualPercent)

3.0165227685434965

In [70]:
def get_price_movement(change):
    if change > 0:
        return 1
    elif change < 0:
        return -1
    else:
        return 0

def get_above_threshold(volatility, threshold):
    if volatility > threshold:
        return True
    else:
        return False

def get_prediction(df):
    if df["new_price_movement"] == 1 and df["new_above_threshold"]:
        return 1 
    elif df["new_price_movement"] == -1 and df["new_above_threshold"]:
        return -1 
    else:
        return 0

In [71]:
df

,Date,Symbol,Exchange,garch,svr,mlp,LSTM,prev_Close,prediction,volatility,ActualDate,ActualClose,ActualPercent,ActualTrend,LSTM_vol,new_vol,new_price_movement,new_above_threshold,new_prediction
0,2022-04-04,AAL,NASDAQ,9.28532,2.479270,2.28121,16.6536,18.23,-1,4.68193,2022-04-05,17.84,2.139331,-1,-8.647285,2.380240,-1,True,-1
1,2022-04-04,AAPL,NASDAQ,2.79687,1.754850,1.60878,159.0210,178.44,-1,2.05350,2022-04-05,175.06,1.894194,0,-10.882650,1.681815,-1,False,0
2,2022-04-04,AMAT,NASDAQ,6.81057,3.116120,3.11230,131.5450,129.44,1,4.34633,2022-04-05,121.71,5.971879,-1,1.626236,3.114210,1,True,1
3,2022-04-04,AMC,NYSE,101.91800,5.439780,5.71697,16.0923,23.31,-1,37.69170,2022-04-05,21.21,9.009009,-1,-30.963964,5.578375,-1,True,-1
4,2022-04-04,AMD,NASDAQ,16.30120,4.396170,4.43730,110.4540,110.53,-1,8.37822,2022-04-05,106.82,3.356555,-1,-0.068760,4.416735,-1,True,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2022-04-04,XHB,AMEX,4.91467,3.107630,3.01361,68.7624,64.39,1,3.67864,2022-04-05,63.06,2.065538,-1,6.790495,3.060620,1,True,1
74,2022-04-04,XLE,AMEX,3.40174,0.995742,0.96538,75.7100,77.16,0,1.78762,2022-04-05,75.95,1.568170,0,-1.879212,0.980561,-1,False,0
75,2022-04-04,XME,AMEX,4.04991,1.475970,1.32960,60.4924,62.07,-1,2.28516,2022-04-05,60.57,2.416626,-1,-2.541647,1.402785,-1,False,0
76,2022-04-04,XOM,NYSE,3.51161,1.200490,1.13034,81.7195,83.16,0,1.94748,2022-04-05,82.73,0.517076,0,-1.732203,1.165415,-1,False,0


In [72]:
df["new_price_movement"] = (df.LSTM - df.prev_Close).apply(get_price_movement)
threshold = 2
df["new_above_threshold"] = df.new_vol.apply(lambda x: get_above_threshold(x, threshold))
df["new_prediction"] = df.apply(get_prediction, axis=1)

In [73]:
df.head()

,Date,Symbol,Exchange,garch,svr,mlp,LSTM,prev_Close,prediction,volatility,ActualDate,ActualClose,ActualPercent,ActualTrend,LSTM_vol,new_vol,new_price_movement,new_above_threshold,new_prediction
0,2022-04-04,AAL,NASDAQ,9.28532,2.47927,2.28121,16.6536,18.23,-1,4.68193,2022-04-05,17.84,2.139331,-1,-8.647285,2.380240,-1,True,-1
1,2022-04-04,AAPL,NASDAQ,2.79687,1.75485,1.60878,159.0210,178.44,-1,2.05350,2022-04-05,175.06,1.894194,0,-10.882650,1.681815,-1,False,0
2,2022-04-04,AMAT,NASDAQ,6.81057,3.11612,3.11230,131.5450,129.44,1,4.34633,2022-04-05,121.71,5.971879,-1,1.626236,3.114210,1,True,1
3,2022-04-04,AMC,NYSE,101.91800,5.43978,5.71697,16.0923,23.31,-1,37.69170,2022-04-05,21.21,9.009009,-1,-30.963964,5.578375,-1,True,-1
4,2022-04-04,AMD,NASDAQ,16.30120,4.39617,4.43730,110.4540,110.53,-1,8.37822,2022-04-05,106.82,3.356555,-1,-0.068760,4.416735,-1,True,-1


## Price Movement

In [78]:
df["actual_price_movement"] = (df.ActualClose - df.prev_Close).apply(get_price_movement)

In [79]:
accuracy_score(df.actual_price_movement, df.new_price_movement, normalize=True)

0.6794871794871795

In [82]:
f1_score(y_true=df.actual_price_movement, y_pred=df.new_price_movement, average='macro')

0.4705403203909856

In [83]:
f1_score(y_true=df.actual_price_movement, y_pred=df.new_price_movement, average='micro')

0.6794871794871795

In [84]:
f1_score(y_true=df.actual_price_movement, y_pred=df.new_price_movement, average='weighted')

0.7008082875582196

## Above Threshold

In [80]:
df["actual_above_threshold"] = df.ActualPercent.apply(lambda x: get_above_threshold(x, threshold))

In [81]:
accuracy_score(df.actual_above_threshold, df.new_above_threshold, normalize=True)

0.7692307692307693

In [85]:
f1_score(y_true=df.actual_above_threshold, y_pred=df.new_above_threshold, average='macro')

0.7678571428571429

In [86]:
f1_score(y_true=df.actual_above_threshold, y_pred=df.new_above_threshold, average='micro')

0.7692307692307693

In [87]:
f1_score(y_true=df.actual_above_threshold, y_pred=df.new_above_threshold, average='weighted')

0.7673992673992674

## Performance Metrics

In [74]:
accuracy_score(df.ActualTrend, df.new_prediction, normalize=True)

0.6666666666666666

In [75]:
f1_score(y_true=df.ActualTrend, y_pred=df.new_prediction, average='macro')

0.46296296296296297

In [76]:
f1_score(y_true=df.ActualTrend, y_pred=df.new_prediction, average='micro')

0.6666666666666666

In [77]:
f1_score(y_true=df.ActualTrend, y_pred=df.new_prediction, average='weighted')

0.6766381766381767